# Police Killings in the US - Data Cleaning in SQL

- The ultimate goal for this project is to use demographics and shooting data to predict the race of victims of fatal police shootings in the US.
- This notebook will focus on the following tasks:
	-  Data cleaning
		- Clean strings
		- Replace null values  
	-  Merging of datasets
	-  Creation and addition of new features
- The following notebook, "SQL_EDA", will focus on exploratory data analysis

---

- The data for this project was loaded into a PostgreSQL database called `police_brutality` using the "db_setup.sql" file. The tables in the database are:
    - `police_killings`
    - `household_income`
    - `poverty_level`
    - `completed_high_school`
    - `race_distribution`
- Data was gathered from two sources:
    - Police Killings Dataset from The Washington Post:
        - Includes every fatal shooting in the US committed by a police officer between 2015 and 2017  
        - Features:
            - Was the victim armed?  
            - How old was the victim?  
            - What is the gender of the victim?  
            - What is the race of the victim?  
            - Where did the shooting take place?  
            - Did the victim appear to be suffering from mental illness?  
            - Did the victim appear to be threatening?  
            - Was the victim trying to flee?  
            - Did the cop have a body camera?
    - [Census Data](https://data.census.gov/cedsci/)
        - Data for over 20,000 cities in the US
        - Features:
            - Racial demographics
            - Median household income
            - Poverty rate
            - Percent of residents over 25 who have completed high school

# Access Database

In [358]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.precision', 3)

In [2]:
plt.style.use('ggplot')
plt.rc('font', size=18)

In [3]:
# Postgres info to connect

connection_args = {
                   'host': 'localhost',  # Connecting to local version of psql
                   'dbname': 'police_brutality',    # DB that we are connecting to
                   'port': 5432          # port opened on AWS
                  }

connection = pg.connect(**connection_args)

In [21]:
cursor = connection.cursor()

# Data Cleaning

## Clean Column Titles

- Switch "geographic_area" to "state" for the following tables so joins can be performed later on:
    - `household_income`
    - `poverty_level`
    - `completed_high_school`
    - `race_distribution`

In [25]:
column_rename = """
                  ALTER TABLE household_income
                RENAME COLUMN geographic_area TO state;
                  ALTER TABLE poverty_level
                RENAME COLUMN geographic_area TO state;
                  ALTER TABLE completed_high_school
                RENAME COLUMN geographic_area TO state;
                  ALTER TABLE race_distribution
                RENAME COLUMN geographic_area TO state;
                """

cursor.execute(column_rename)
cursor.execute('commit;')

---

In [267]:
# cursor.execute('rollback;')

In [324]:
# drop_views = """    
#               DROP VIEW v_demographics_clean;
#               DROP VIEW v_demographics;
#               DROP VIEW v_poverty_level_clean;
#               DROP VIEW v_completed_high_school_clean;
#               DROP VIEW v_race_distribution_clean;
#               DROP VIEW v_race_distribution;
#               DROP VIEW v_household_income_clean;
#               """

# cursor.execute(drop_views)
# cursor.execute('commit;')

---

## Clean City Columns and Fix Data Types

- The demographic tables have the geographic type tacked on the end of each city (e.g. city, town, CDP, etc.)
    - These will need to be removed and ultimately extracted to be used as a separate feature.
- The following features are strings and need to be converted to a numeric data type:
    - `poverty_rate`
    - `percent_completed_hs`
    - `share_white`
    - `share_black`
    - `share_native_american`
    - `share_asian`
    - `share_hispanic`
- A merge key will be created out of city + state so that these tables can be merged with the `police_killings` table

In [325]:
clean_views = """
              CREATE VIEW v_poverty_level_clean AS
                   SELECT CONCAT(REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_'), '_', state) AS merge_key,
                          INITCAP(REGEXP_REPLACE(city, '^.* ', '')) AS city_type,
                          REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_') AS city,
                          state,
                          CASE WHEN poverty_rate = '-' THEN NULL
                          ELSE CAST(poverty_rate as numeric)
                           END AS poverty_rate
                     FROM poverty_level;
                     
              CREATE VIEW v_completed_high_school_clean AS
                   SELECT CONCAT(REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_'), '_', state) AS merge_key,
                          INITCAP(REGEXP_REPLACE(city, '^.* ', '')) AS city_type,
                          REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_') AS city,
                          state,
                          CASE WHEN percent_completed_hs = '-' THEN NULL
                          ELSE CAST(percent_completed_hs as numeric)
                           END AS percent_completed_hs
                     FROM completed_high_school;
                     
              CREATE VIEW v_race_distribution AS
                   SELECT CONCAT(REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_'), '_', state) AS merge_key,
                          INITCAP(REGEXP_REPLACE(city, '^.* ', '')) AS city_type,
                          REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_') AS city,
                          state,
                          CASE WHEN share_white = '(X)' THEN NULL
                          ELSE CAST(share_white AS numeric)
                           END AS share_white,
                          CASE WHEN share_black = '(X)' THEN NULL
                          ELSE CAST(share_black AS numeric)
                           END AS share_black,
                          CASE WHEN share_native_american = '(X)' THEN NULL
                          ELSE CAST(share_native_american AS numeric)
                           END AS share_native_american,
                          CASE WHEN share_asian = '(X)' THEN NULL
                          ELSE CAST(share_asian AS numeric)
                           END AS share_asian,
                          CASE WHEN share_hispanic = '(X)' THEN NULL
                          ELSE CAST(share_hispanic AS numeric)
                           END AS share_hispanic
                     FROM race_distribution;
                     
              CREATE VIEW v_household_income_clean AS
                   SELECT CONCAT(REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_'), '_', state) AS merge_key,
                          INITCAP(REGEXP_REPLACE(city, '^.* ', '')) AS city_type,
                          REPLACE(TRIM(REGEXP_REPLACE(city, '\S+$', '')), ' ', '_') AS city,
                          state,
                          CASE WHEN median_income = '-' or median_income = '(X)' THEN NULL
                          ELSE TRIM(TRANSLATE(median_income, '-,+', ' , '))
                           END AS median_income
                     FROM household_income;               
              """

cursor.execute(clean_views)
cursor.execute('commit;')

## Remove Duplicates in `race_distribution` Table

- Some cities have duplicate entries for demographics (listed under multiple geographic types, e.g. city, CDP, etc.)
    - The duplicates need to be removed.

In [326]:
race_view = """
             CREATE VIEW v_race_distribution_clean AS
               WITH preview AS (
                              SELECT merge_key,
                                     city_type,
                                     city,
                                     state,
                                     share_white,
                                     share_black,
                                     share_native_american,
                                     ROW_NUMBER() OVER(PARTITION BY merge_key ORDER BY share_native_american) AS row_number,
                                     share_asian,
                                     share_hispanic
                                FROM v_race_distribution
                             )
             SELECT merge_key,
                    city_type,
                    city,
                    state,
                    share_white,
                    share_black,
                    share_native_american,
                    share_asian,
                    share_hispanic
               FROM preview
              WHERE row_number = 1;
             """

cursor.execute(race_view)
cursor.execute('commit;')

## Merge Demographic Tables into One Table

- The following tables will be merged:
    - `race_distribution`
    - `poverty_level`
    - `completed_high_school`
    - `household_income`

In [327]:
demographic_view = """
                   CREATE VIEW v_demographics AS
                        SELECT r.merge_key,
                               r.city_type,
                               r.city,
                               r.state,
                               p.poverty_rate,
                               s.percent_completed_hs,
                               r.share_white,
                               r.share_black,
                               r.share_native_american,
                               r.share_asian,
                               r.share_hispanic,
                               CASE WHEN i.median_income IS NULL THEN NULL
                               ELSE CAST(REPLACE(i.median_income, ',', '') AS int)
                                END AS median_income
                          FROM v_race_distribution_clean r
               FULL OUTER JOIN v_poverty_level_clean p
                            ON r.merge_key = p.merge_key
               FULL OUTER JOIN v_completed_high_school_clean s
                            ON r.merge_key = s.merge_key
               FULL OUTER JOIN v_household_income_clean i
                            ON r.merge_key = i.merge_key;
                   """

cursor.execute(demographic_view)
cursor.execute('commit;')

## Clean Up "City Type" feature
- For example, 'County' is listed multiple ways ((County), Counties, County, etc.)
    - These need to be consolidated

In [328]:
demo_clean = """
             CREATE VIEW v_demographics_clean AS         
                  SELECT merge_key,
                         CASE WHEN city_type ILIKE 'Count%' THEN 'County'
                              WHEN city_type IN ('', 'Bow') THEN 'City'
                              WHEN city_type ILIKE '%Balance%' THEN 'Government'
                              ELSE city_type 
                            END AS city_type,
                         city,
                         state,
                         poverty_rate,
                         percent_completed_hs,
                         share_white,
                         share_black,
                         share_native_american,
                         share_asian,
                         share_hispanic,
                         median_income
                    FROM v_demographics;
             """

cursor.execute(demo_clean)
cursor.execute('commit;')

In [329]:
query = """
        SELECT * 
          FROM v_demographics_clean;
        """

demo_clean = pd_sql.read_sql(query, connection)

In [330]:
demo_clean[demo_clean.merge_key.isna()]

merge_key city_type  city state  poverty_rate  percent_completed_hs  \
71         None      None  None  None           NaN                  89.7   
72         None      None  None  None           NaN                  77.1   
73         None      None  None  None           NaN                  95.7   
74         None      None  None  None           NaN                  52.6   
75         None      None  None  None           NaN                  86.5   
...         ...       ...   ...   ...           ...                   ...   
29908      None      None  None  None           NaN                   NaN   
29976      None      None  None  None           NaN                   NaN   
30029      None      None  None  None           NaN                   NaN   
30099      None      None  None  None           NaN                   NaN   
30196      None      None  None  None           NaN                   NaN   

       share_white  share_black  share_native_american  share_asian  \
71             NaN          NaN                    NaN          NaN   
72             NaN          NaN                    NaN          NaN   
73             NaN          NaN                    NaN          NaN   
74             NaN          NaN                    NaN          NaN   
75             NaN          NaN                    NaN          NaN   
...            ...          ...                    ...          ...   
29908          NaN          NaN                    NaN          NaN   
29976          NaN          NaN                    NaN          NaN   
30029          NaN          NaN                    NaN          NaN   
30099          NaN          NaN                    NaN          NaN   
30196          NaN          NaN                    NaN          NaN   

       share_hispanic  median_income  
71                NaN            NaN  
72                NaN            NaN  
73                NaN            NaN  
74                NaN            NaN  
75                NaN            NaN  
...               ...            ...  
29908             NaN        72500.0  
29976             NaN        40441.0  
30029             NaN        43875.0  
30099             NaN            NaN  
30196             NaN            NaN  

[429 rows x 12 columns]

In [343]:
random_query = """
                 SELECT *
                   FROM v_demographics_clean
                  WHERE merge_key = 'Fort_Valley_GA';
               """

pd_sql.read_sql(random_query, connection)

merge_key city_type         city state  poverty_rate  \
0  Fort_Valley_GA      City  Fort_Valley    GA          45.2   

   percent_completed_hs  share_white  share_black  share_native_american  \
0                  74.1         13.6         81.6                    0.2   

   share_asian  share_hispanic  median_income  
0          0.3             5.8          20962

## Merge `police_killings` with `v_demographics_clean`

- This query accomplishes the following tasks:
    - **Merges `police_killings` table with demographics data**
    - **Replaces null values**
        - For `armed`: replace with 'Undetermined'
        - For `age`: replace with median age of corresponding state
        - For `race`: replace with 'Not Identified`
        - For `flee`: replace with 'Not Applicable'
        - For demographics: replace with nearby city values
    - **Creates/Updates several features:**
        - `region`: consolidates states into 7 US regions
        - `year`: year of police killing
        - `month`: month of police killing
        - `day_of_week`: day of week of police killing
        - `armed`: consolidates various descriptions into 7 categories
    

In [352]:
merge_query = """
              CREATE VIEW demo_killings AS
                     WITH median_age AS (
                                           SELECT state,
                                                  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY age) AS age
                                             FROM police_killings
                                         GROUP BY state
                                        ),
                 median_income_state AS (
                                           SELECT state,
                                                  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY median_income) AS median_income
                                             FROM v_demographics_clean
                                         GROUP BY state
                                        )                                   

              SELECT p.city,
                     p.state,
                     d.city_type,
                     CASE WHEN p.state IN ('CA', 'OR', 'WA') THEN 'West Coast'
                          WHEN p.state IN ('ID', 'MT', 'WY', 'CO', 'NV', 'UT', 'AZ', 'NM') THEN 'Mountain'
                          WHEN p.state IN ('ND', 'SD', 'NE', 'KS', 'IA') THEN 'Central'
                          WHEN p.state IN ('MN', 'WI', 'MI', 'IL', 'MO', 'IN', 'OH') THEN 'Midwest'
                          WHEN p.state IN ('OK', 'TX', 'AR', 'LA', 'MS', 'AL', 'FL', 'GA', 'SC', 'NC', 'KY', 'TN', 'VA', 'WV') THEN 'South'
                          WHEN p.state IN ('HI', 'AK') THEN 'Other'
                          ELSE 'Northeast'
                          END AS region,
                     DATE_PART('year', TO_DATE(p.date, 'DD/MM/YY')):: int AS year,
                     TO_CHAR(TO_DATE(p.date, 'DD/MM/YY'),'Month') AS month,
                     TO_CHAR(TO_DATE(p.date, 'DD/MM/YY'), 'Day') AS day_of_week,
                     p.manner_of_death,
                     CASE WHEN p.armed = 'unarmed' THEN p.armed
                          WHEN p.armed IN ('gun', 'crossbow', 'gun and knife', 'guns and explosives', 'hatchet and gun', 'machete and gun', 'bean-bag gun', 'nail gun') THEN 'gun'
                          WHEN p.armed IN ('Taser', 'chainsaw', 'hand torch') THEN 'gun'
                          WHEN p.armed IN ('knife', 'machete', 'ax', 'sword', 'box cutter', 'hatchet', 'sharp object', 'pole and knife', 'straight edge razor', 'glass shard', 'lawn mower blade') THEN 'sharp object'
                          WHEN p.armed IN ('screwdriver', 'scissors', 'metal stick', 'metal rake', 'pitchfork', 'pick-axe', 'baseball bat and fireplace poker', 'garden tool', 'crowbar', 'spear', 'bayonet', 'carjack', 'metal hand tool', 'cordless drill') THEN 'sharp object'
                          WHEN p.armed IN ('baseball bat', 'metal pipe', 'hammer', 'blunt object', 'rock', 'brick', 'metal object', 'piece of wood', 'beer bottle', 'metal pole', 'meat cleaver', 'pipe', 'baton', 'shovel', 'stapler', 'contractor level', 'chain', 'flagpole', 'baseball bat and bottle', 'pole', 'air conditioner', 'tire iron', 'oar', 'fireworks') THEN 'blunt object'
                          WHEN p.armed IN ('toy weapon', 'pen', 'flashlight') THEN 'non-lethal object'
                          WHEN p.armed IN ('vehicle', 'motorcycle') THEN 'vehicle'
                          ELSE 'undetermined' END AS armed,
                     CASE WHEN p.age IS NULL THEN (SELECT median_age.age 
                                                     FROM median_age 
                                                    WHERE p.state = median_age.state)
                          ELSE p.age END AS age,
                     p.gender,
                     CASE WHEN p.race IS NULL THEN 'Not Identified'
                          ELSE p.race END AS race,
                     p.signs_of_mental_illness,
                     p.threat_level,
                     CASE WHEN p.flee IS NULL THEN 'Not Applicable'
                          ELSE p.flee END AS flee,
                     p.body_camera,
                     CASE WHEN p.city = 'Macon' AND p.state = 'GA' THEN (SELECT poverty_rate
                                                                           FROM v_demographics_clean
                                                                          WHERE merge_key = 'Fort_Valley_GA')
                          ELSE d.poverty_rate END AS poverty_rate,
                     CASE WHEN p.city = 'Macon' AND p.state = 'GA' THEN (SELECT percent_completed_hs
                                                                           FROM v_demographics_clean
                                                                          WHERE merge_key = 'Fort_Valley_GA')
                          ELSE d.percent_completed_hs END AS percent_completed_hs,
                     d.share_white,
                     d.share_black,
                     d.share_native_american,
                     d.share_asian,
                     d.share_hispanic,
                     CASE WHEN p.city = 'Macon' AND p.state = 'GA' THEN (SELECT median_income
                                                                           FROM v_demographics_clean
                                                                          WHERE merge_key = 'Fort_Valley_GA')
                          WHEN p.city = 'Indianapolis' AND p.state = 'IN' THEN (SELECT median_income
                                                                                  FROM v_demographics_clean
                                                                                 WHERE merge_key = 'Beech_Grove_IN')
                          WHEN d.median_income IS NULL THEN (SELECT median_income_state.median_income 
                                                               FROM median_income_state
                                                              WHERE p.state = median_income_state.state)
                          ELSE d.median_income END AS median_income
                FROM police_killings p
           LEFT JOIN v_demographics_clean d
                  ON CONCAT(TRIM(REPLACE(p.city, ' ', '_')), '_', p.state) = d.merge_key
           WHERE share_white IS NOT NULL;
           """

cursor.execute(merge_query)
cursor.execute('commit;')


In [355]:
random_query = """
                 SELECT *
                   FROM demo_killings;
               """

demo_killings_df = pd_sql.read_sql(random_query, connection)

# Save Final Table as Dataframe

- For use in "SQL_EDA" notebook

In [359]:
# with open('Data/sql_demo_killings_df.pkl', 'wb') as to_write:
#     pickle.dump(demo_killings_df, to_write)